# Collecting ASCAP Info + Storing in DB

**Description**: Pulling necessary song info from the ASCAP HTML files & storing in PostreSQL db

In [27]:
import json
import os
import pickle
import re

from bs4 import BeautifulSoup, element
from sqlalchemy import create_engine

import numpy as np
import pandas as pd

## Loading in HTML File

I'll need to figure out how to retrieve all of the corresponding song details from each HTML file that I scraped from ASCAP>

In [37]:
with open('../data/ascap_songs/CHIEF KEEF.html', 'rb+') as f:
    test_html = BeautifulSoup(f, 'lxml')

### Retrieving Song Cards

My current thought is that I should divvy these results into a list of `dict`s, which should allow me to easily toss into a dataframe or db afterwards.

In [38]:
test_cards = test_html.find_all('div', {'class':'card hyphenate card--expanded'})

64 songs indexed for Chief Keef

In [39]:
len(test_cards)

64

In [45]:
tc2 = test_cards[2]

In [57]:
tc2.find('div', {'class':'workID'}).text.strip().split()[-1]

'887415977'

##### The title and "Total Current ASCAP Share"

In [13]:
test_cards[0].find('div', {'class':'card__header__title'}).text.strip().split(sep='\n')

['ADAM & EVE',
 '                ',
 '                    ',
 '                    Total Current ASCAP Share:',
 '                    25%']

##### Just the song title

In [14]:
test_cards[0].find('div', {'class':'card__header__title'}).text.strip().split(sep='\n')[0]

'ADAM & EVE'

##### Writers

In [15]:
writers_row = test_cards[0].find_all('tr', {'class':'writers__row'})

In [16]:
writer_list = []

for writer in writers_row:
    if isinstance(writer.find('a'), element.Tag):
        writer_list.append((writer.find('a'), writer.find_all('td', {'class':'sup-col'})))
    elif isinstance(writer.find('span'), element.Tag):
        writer_list.append((writer.find('span'), writer.find_all('td', {'class':'sup-col'})))
    else:
        writer_list.append('cannot find writer')
        
writer_info = []

for writer, writer_ids in writer_list:
    b = writer.attrs
    b['pro'] = writer_ids[0].text.strip()
    b['ipi'] = writer_ids[1].text.strip()
    writer_info.append(b)

In [17]:
writer_list[0][0].attrs.pop('href')
writer_list[0][0].attrs.pop('class')
writer_list[0][0].attrs

{'data-writer': 'MCKINNEY REGINALD CHRISTOPHER',
 'data-id': '652264355',
 'pro': 'ASCAP',
 'ipi': '652264355'}

In [18]:
writer_info

[{'data-writer': 'MCKINNEY REGINALD CHRISTOPHER',
  'data-id': '652264355',
  'pro': 'ASCAP',
  'ipi': '652264355'},
 {'class': ['no-click'],
  'data-writer': 'WRITER UNKNOWN  ',
  'data-id': '0',
  'pro': 'NS',
  'ipi': '—'}]

Performers

In [19]:
performers = test_cards[0].find_all('div', {'class':'creditors__list'})

In [20]:
performers[0].text.strip('\n')

'CHIEF KEEF'

Retreive one or multiple performers per song

In [21]:
if performers[0].text.count('\n') > 2:
    performers[0].text.strip().split(sep='\n')
else:
    performers[0].text.strip('\n')

HTML Title

### Defining Functions to Sort Through HTML Files

Here's how I'll take the song information that I need and output it into a list of `dict`s

In [60]:
def bs_file(file):
    '''
    Opens file and returns beautiful soup object
    '''
    with open('../data/ascap_songs/{}'.format(file), 'rb') as f:
        bs = BeautifulSoup(f, 'lxml')
    return bs

def song_sorter(bs):
    '''
    Iterates through songs in bs object and returns cards of all songs
    '''
    results = []
    if len(bs.find_all('div', {'class':'card hyphenate card--expanded'})) > 0:
        results.extend(bs.find_all('div', {'class':'card hyphenate card--expanded'}))
    if len(bs.find_all('div', {'class':'card card--expanded hyphenate'})) > 0:
        results.extend(bs.find_all('div', {'class':'card card--expanded hyphenate'}))
    return results
                                   
def get_song_credits(card):
    '''
    Retrieves song credits from an individual song card
    '''   
    title = card.find('div', {'class':'card__header__title'}).text.strip().split(sep='\n')[0]
    aid = card.find('div', {'class':'workID'}).text.strip().split()[-1]
    writers_row = card.find_all('tr', {'class':'writers__row'})
    writer_list = []
    
    # Creating writer list for further filtering
    for writer in writers_row:
        if isinstance(writer.find('a'), element.Tag):
            writer_list.append((writer.find('a'), writer.find_all('td', {'class':'sup-col'})))
        elif isinstance(writer.find('span'), element.Tag):
            writer_list.append((writer.find('span'), writer.find_all('td', {'class':'sup-col'})))
        else:
            writer_list.append('cannot find writer')
    writers = []
    for writer, writer_ids in writer_list:
        b = writer.attrs
        b['pro'] = writer_ids[0].text.strip()
        b['ipi'] = writer_ids[1].text.strip()
        writers.append(b)
    
    # Grabbing performers
    performers = card.find_all('div', {'class':'creditors__list'})
    if performers[0].text.count('\n') > 2:
        artists = performers[0].text.strip().split(sep='\n')
    else:
        artists = performers[0].text.strip('\n')
        
    # Grabbing alternate titles
    try:
        if performers[1].text.count('\n'):
            alt_titles = performers[1].text.strip().split(sep='\n')
        else:
            alt_titles = performers[1].text.strip('\n')
    except IndexError:
        alt_titles = None
        
    return title, aid, writers, artists, alt_titles

In [69]:
ck = song_sorter(test_html)
t, aid, w, a, at = get_song_credits(ck[2])

In [70]:
aid

'887415977'

In [46]:
len(ck)

64

In [71]:
def get_html_songs(html_list):
    '''
    Retreives songs in html files and stores in a list of dicts
    '''
    song_list = []
    count = 0
    for file in html_list:
        bs = bs_file(file)
        songs = song_sorter(bs)
        for song in songs:
            count += 1
            t, aid, w, p, at = get_song_credits(song)
            song_dict = {}
            song_dict['Title'] = t
            song_dict['AID'] = aid
            song_dict['Writers'] = w
            song_dict['Performers'] = p
            song_dict['Alt-Titles'] = at
            song_list.append(song_dict)
            if len(song_list) % 5000 == 0:
                print('{} songs completed'.format(count))
    return song_list

## Retrieving HTML File List

In [5]:
html_list = os.listdir('../data/ascap_songs/')

In [6]:
len(html_list)

13019

In [7]:
html_list.index('.ipynb_checkpoints')

9917

In [8]:
html_list.pop(9917)

'.ipynb_checkpoints'

#### Ensuring there are no more non-html files in list

In [14]:
files = [html.find('.html') for html in html_list]
files.sort()
files[:10]

[0, 1, 2, 2, 2, 2, 2, 2, 2, 2]

### Quick Function Test

In [13]:
test = get_html_songs(html_list[0:5])

In [63]:
html_list[0:5]

['BONE THUGS N HARMONY FEAT. EAZY-E.html',
 'VESTAL AND LEE GREENWOOD.html',
 ' PETE SEEGER.html',
 'DARIUS RUCKER FEAT JILL SCOTT.html',
 'GARTH BROOKS.html']

In [14]:
len(test)

7

## Retrieving Song Results

In [72]:
song_list = get_html_songs(html_list)

5000 songs completed
10000 songs completed
15000 songs completed
20000 songs completed
25000 songs completed
30000 songs completed
35000 songs completed
40000 songs completed
45000 songs completed
50000 songs completed
55000 songs completed
60000 songs completed
65000 songs completed
70000 songs completed
75000 songs completed
80000 songs completed
85000 songs completed
90000 songs completed
95000 songs completed
100000 songs completed
105000 songs completed
110000 songs completed
115000 songs completed
120000 songs completed
125000 songs completed
130000 songs completed
135000 songs completed
140000 songs completed
145000 songs completed
150000 songs completed
155000 songs completed
160000 songs completed
165000 songs completed
170000 songs completed
175000 songs completed
180000 songs completed
185000 songs completed
190000 songs completed
195000 songs completed
200000 songs completed
205000 songs completed
210000 songs completed
215000 songs completed
220000 songs completed
225000 s

#### Saving `song_list` as pickle

In [73]:
with open('../pickle/ascap_credits.pkl', 'wb') as f:
    pickle.dump(song_list, f)

### Examining `song_list`

In [77]:
song_list[129020]

{'Title': "BOSSIN' UP",
 'AID': '886044026',
 'Writers': [{'class': ['no-click'],
   'data-writer': 'COLLINS BRIAN ',
   'data-id': '0',
   'pro': 'NS',
   'ipi': '—'},
  {'class': ['no-click'],
   'data-writer': 'FERGUSON DAROLD ',
   'data-id': '0',
   'pro': 'NS',
   'ipi': '—'},
  {'class': ['no-click'],
   'data-writer': 'KHARBOUCH KARIM ',
   'data-id': '0',
   'pro': 'NS',
   'ipi': '—'},
  {'href': 'javascript:void(0)',
   'class': ['name'],
   'data-writer': 'TAFT STEPAN ',
   'data-id': '539311163',
   'pro': 'ASCAP',
   'ipi': '539311163'},
  {'class': ['no-click'],
   'data-writer': 'THOMAS JAMES ',
   'data-id': '0',
   'pro': 'NS',
   'ipi': '—'}],
 'Performers': 'KID INK',
 'Alt-Titles': None}

In [17]:
len(song_list)

309581

#### Loading List into DataFrame

In [78]:
df = pd.DataFrame(data=song_list)

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309581 entries, 0 to 309580
Data columns (total 5 columns):
AID           309581 non-null object
Alt-Titles    94080 non-null object
Performers    309581 non-null object
Title         309581 non-null object
Writers       309581 non-null object
dtypes: object(5)
memory usage: 11.8+ MB


In [80]:
df.head()

,AID,Alt-Titles,Performers,Title,Writers
0,360318916,"[FOE THA LOVE OF $, FOE THA LOVE OF $, FOE THA...","[BONE, BONE THUGS N HARMONY, BONE THUGS N HARM...",FOR THA LOVE OF MONEY,"[{'class': ['no-click'], 'data-writer': 'CARRA..."
1,530659306,None,"[GOODMAN V, GREENWOOD AND VESTAL, GREENWOOD L,...",WE THE PEOPLE,"[{'href': 'javascript:void(0)', 'class': ['nam..."
2,334030418,[CELERY TIME],ARLO GUTHRIE / PETE SEEGER,CELERY-TIME,"[{'href': 'javascript:void(0)', 'class': ['nam..."
3,442081954,None,ARLO GUTHRIE / PETE SEEGER,NEUTRON BOMB,"[{'href': 'javascript:void(0)', 'class': ['nam..."
4,230055482,None,"[ARLO GUTHRIE / PETE SEEGER, GUTHRIE A, GUTHRI...",WILL THE CIRCLE BE UNBROKEN,"[{'href': 'javascript:void(0)', 'class': ['nam..."


#### Deleting potential duplicates

Duplicate 'AID' records should be the only song duplicates

In [89]:
df[df.duplicated('AID')].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45406 entries, 534 to 309580
Data columns (total 5 columns):
AID           45406 non-null object
Alt-Titles    28281 non-null object
Performers    45406 non-null object
Title         45406 non-null object
Writers       45406 non-null object
dtypes: object(5)
memory usage: 2.1+ MB


In [86]:
len(dupes)

309581